## Upload personal data to database
## ( the best version )

#### Import the libraries to get the env variables

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [3]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

#### Set the name of the database

In [4]:
database = "./electrotehnica_documents/"

#### Import the libraries to get the text and split it in chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader

c:\Users\uif94707\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


##### Get the text from the documents

In [6]:
# loader = DirectoryLoader(database, glob="./*.txt", loader_cls=TextLoader)
loader = DirectoryLoader(database, glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
documents


[Document(page_content='ELEMENTELE IDEALE DE CIRCUIT ÎN REGIM SINUSOIDAL  \n \n Rezistorul ideal   \nRezistorul ideal prezintă o rezistenţă electrică R şi este caracterizat de ecuaţia:  \n)( u(t) tRi=\n \nEcuaţia de funcţionare în complex devine:  \nIRU=\n \nImpedanţa complexă a rezistorului va fi:  \nIUZ=\n  \nDefazajul dintre tensiune şi curent este nul:  \n\uf06a = 0   \nÎn figura este reprezentat simbolul rezistorului, variaţia în timp a tensiunii şi curentului \nşi diagrama fazorială.  \n \n \n \nBobina ideală   \nBobina ideal ă prezintă numai inductivitatea L (rezistenţa proprie este egală cu zero).  \nEcuaţia de funcţionare este:  \ndtdi(t)L)(uL=t\n  \nEcuaţia de funcţionare  în complex:  \nU=j\n\uf077LI  \nImpedanţa complexă:  \nZ=j\n\uf077L  ', metadata={'source': 'electrotehnica_documents\\curs1.pdf', 'page': 0}),
 Document(page_content='Modulul impedanţei complexe \nL X ZL\uf077==  - se numeşte reactanţa inductivă. Tensiunea la \nbornele bobinei este defazată cu \n2π=\uf06a 

##### Split the text into chunks

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)

len(chunks)

2

#### Import the libraries to connect to Chroma database

In [9]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [10]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [11]:
# Given string
input_string = "./electrotehnica_documents/"

# Split the string by '/'
parts = input_string.split('/')

# Get the last part
desired_output = parts[-2]

print(desired_output)

electrotehnica_documents


##### Upload the text in chroma database

In [12]:
import re

result = re.search(r'(?P<word>\w+)_', desired_output)

if result:
    extracted_word = result.group('word')
    print(extracted_word)

database_name = extracted_word+"-database"
database_name

electrotehnica


'electrotehnica-database'

In [14]:
db2 = Chroma.from_documents(chunks, embedding_function, persist_directory=database_name)